# Searching EarthCODE with the OSC library

This notebook walks through how to use the EarthCODE OSC search function and access OSC records programatically using the `earthcode` python library. The examples you see below are live metadata records that live on the [Open Science Catalog](https://opensciencedata.esa.int/). Using the search functionality you can find relevant `productds`, `variables`, `eo-missions`, and `projects` using various filters on the metadata as well as geographical constraints.

With this search you can directly use natural language for your queries - passing it an abstract, idea or an exhurb from your research to see what's relevant for you in the Open Science Catalog.

If you are a scientist publishing to the EarthCODE OSC you can get relevant suggestions for variables, themes or missions for your dataset.

There are different ways of searching for data - the simplest one is searching with a natural language query. The results will be returned as a list of pystac collection (for `products`) or catalog objects.

In [1]:
from earthcode.search import search
# note: In the current version of the osc library - the first run of the library right after you install it, searching will prefetch a mini-llm model which should take approximately 1 minute

search("forest fires")[0]


/Users/dean/Documents/EarthCODE/osc_test/.pixi/envs/default/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Collection id=seasfire-cube>

You can also fetch directly a known collection by its ID if you already know it. This keeps results tight and avoids scanning the whole catalog. You can either pass one id as a `str` or multiple ids as a list `List[str]`. In both cases the result will be an array object.

For example, here we've taken the id of the search result above to fetch it directly by its id instead of a semantic search:


In [2]:
search(collection_ids="seasfire-cube")


[<Collection id=seasfire-cube>]

You can also use `bbox=[minx, miny, maxx, maxy]` to limit by geography. By default the bbox only needs to intersect the product footprint. For example, a small local area bbox will always intersect global datasets. In the below example we search for snow data over the Alps.

Additionally, we can set a limit on the amount of results we get back using the `limit` parameter as in the example below.

Note that the result is a global dataset which intersects with our bbox for the Alps.

In [3]:
alps_bbox = [5.95591129, 45.81799493, 10.49229402, 47.80846475]
search("snow data", limit=1, bbox=alps_bbox)[0]


<Collection id=binary-wet-snow-s14science-snow>

If instead you need the whole footprint to sit inside the bbox (containment instead of overlap) use `intersects=False`.

For example, in the cell below we run the same query again but with `intersects=False` and get results that are only within the Alps (excluding global ones and others that overlap).

In [4]:
search("snow data", limit=1, bbox=alps_bbox, intersects=False)[0]

<Collection id=snow-cover-alpglacier>

## Searching for Variables

You can also search for variables to find relevant parameters or measured quantities. The search works in the same way, although there is no filter for bbox or filter for eo-missions. You can search for terms directly using natural language as in the example below:

In [5]:
chlorophyll = search("chlorophyll", type="variables", limit=2)[0]
chlorophyll

<Catalog id=chlorophyll-concentration>

You can use the variable search to help with approximations about what variables to use for your published dataset by feeding the description of your project to the search - and avoid creating new variables which already might exist in the catalog. For example the below example searches variables that migth fit this product: https://opensciencedata.esa.int/products/ice-sheet-velocity-antarctic-2021/collection.


*'This dataset contains monthly gridded ice velocity maps of the Antarctic Ice Sheet derived from Sentinel-1 data acquired between 2021-01-01 and 2021-12-31. It was generated by ENVEO, as part of the ESA Antarctic Ice Sheet Climate Change Initiative project (Antarctic_Ice_Sheet_cci). The surface velocity is derived by applying feature tracking techniques using Sentinel-1 synthetic aperture radar (SAR) data acquired in the Interferometric Wide (IW) swath mode. Ice velocity is provided at 200m grid spacing in Polar Stereographic projection (EPSG: 3031). The horizontal velocity components are provided in true meters per day, towards easting and northing direction of the grid. The vertical displacement is derived from a digital elevation model. Provided is a NetCDF file with the velocity components: vx, vy, vz, along with maps showing the magnitude of the horizontal components, the valid pixel count and uncertainty. The product combines all ice velocity maps, based on 6- and 12-day repeats, acquired within a single month in a monthly averaged product.'*

In [6]:
product_description = search(collection_ids="ice-sheet-velocity-antarctic-2021")[0].description

result = search(product_description, type="variables", limit=5)

[r.title for r in result]

['Ice sheet topography',
 'Ice Temperature',
 'Ice Velocity',
 'Sea Ice Age',
 'Glacier motion']

In much the same way as variables you can search for EO missions to see which platforms match your research as in the example below:

In [7]:
search("sentinel-1", type="eo-missions", limit=2)[0]


<Catalog id=sentinel-1>

# Filtering Products by variables, missions and themes

Once you’ve identified a `variable` record (for example, `chlorophyll` from the Variables section), you can use its `id` to filter **products** to only those that explicitly declare that variable in their metadata. This is useful when you want results that are semantically relevant *and* models a particular variable of interest.

In the example below, we search for global chlorophyll datasets and constrain results to products that include the chlorophyll variable:


In [8]:
search("global chlorophyll dataset", variable=chlorophyll.id, type="products")

[<Collection id=monthly-global-chlorophyll-a-dataset-9-km-oc-cci-v42>,
 <Collection id=global-particulate-organic-carbon-v5-bicep>,
 <Collection id=l4-chla-eo4sibs>,
 <Collection id=l3-chla-daily-eo4sibs>,
 <Collection id=4dmed-3d-prim-prod-150>,
 <Collection id=high-res-phytoplankton-chl-a-data-at-discrete-light-stations-polarstern-cruise-ps113>,
 <Collection id=phytoplankton-group-chl-ll-using-the-mean-dac-over-the-first-optical-depth-kdmean-of-the-measured-radiometric-profile>,
 <Collection id=coastal-carbon-mapper-reflectance-water-quality-data>,
 <Collection id=4dmed-t-s-geo-a-150>,
 <Collection id=secchi-disk-depth-lpf-physioglob>]

You can also filter by eo-missions (e.g. sentinel-1), to find products that have been produced by that instrument. The queries are addative so you can tighten the same query above further by restricting it to a specific EO mission. This is particularly helpful when multiple missions produce similar variables (e.g., ocean colour chlorophyll from different instruments), but you only want products traceable to a given platform. 

In [9]:
search("global chlorophyll dataset", variable=chlorophyll.id, type="products", mission="sentinel-3")

[<Collection id=monthly-global-chlorophyll-a-dataset-9-km-oc-cci-v42>,
 <Collection id=global-particulate-organic-carbon-v5-bicep>,
 <Collection id=l4-chla-eo4sibs>,
 <Collection id=l3-chla-daily-eo4sibs>]

You can also add a free-text keyword search which finds all literal occurences of the phrase/word you've passed in the title, description or keyword entries of the product you are looking for. Here we narrow to results that mention “daily”, while still keeping the variable and mission constraints:

In [10]:
search("global chlorophyll dataset", variable=chlorophyll.id, type="products", mission="sentinel-3", keyword="daily")

[<Collection id=l4-chla-eo4sibs>, <Collection id=l3-chla-daily-eo4sibs>]

You can also refine variable search by applying the filters for `theme` and `keyword`. This is especially useful when the catalog contains many similar parameters and you want to narrow results to a particular scientific domain or concept.

For example, the query below searches for a variable related to jet streams, but restricts results to the *oceans* theme and further filters by the keyword *cyclone*:

In [11]:
results = search(keyword="seasonal fire modeling", type="products", limit=5)
ids = {r.id for r in results}

print(ids)

{'seasfire-cube'}


In [12]:
search("find me a variable that talks about jet streams",theme="oceans", type="variables", keyword="cyclone")[0].title

'East Atlantic Jet Pattern'